In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
from torchvision import datasets as tvds
from torchvision import transforms
from tqdm.notebook import tqdm
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
# device = "cpu"
torch.cuda.empty_cache()

print(device)

cuda


## Read Image dataset
- Download the dataset
- Add transformer

In [3]:
im_transformer = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.RandomAdjustSharpness(1.2),
        transforms.RandomVerticalFlip(),
        transforms.RandomGrayscale(),
        transforms.RandomRotation(degrees=180),
        transforms.Resize(150),
        transforms.ToTensor(),
        transforms.Normalize((.5, .5, .5), (.5, .5, .5)),
#         transforms.Lambda(lambda x: collate_fn(x))
    ]
)

In [4]:
train_dataset = tvds.CIFAR100("./cifar", transform=im_transformer, download=True, train=True)
test_dataset = tvds.CIFAR100("./cifar", transform=im_transformer, download=True, train=False)

Files already downloaded and verified
Files already downloaded and verified



There is a structure issue, need to fix it.

Current structure
[
    {'x": "", "y": ""}
]

Required Structure
{"x": [""], "y": [""]}

In [5]:
def collate_fn(batch):
    x_, y_ = [], []
    for (x, y) in batch:
        x_.append(x)
        y_.append(y)
    return {
        "x": torch.stack(x_).to(device),
        "y": torch.from_numpy(np.array(y_, dtype=np.int64)).to(device)
    }

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=2, collate_fn=collate_fn)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=1, collate_fn=collate_fn)

In [8]:
class ClassificationModel(nn.Module):
    
    def __init__(self, out_dim, channel_first = False):
        super(ClassificationModel, self).__init__()
        self.input = nn.Conv2d(3, 256, (3, 3), padding="valid")
        self.conv1 = nn.Conv2d(256, 128, (3, 3))
        self.maxp1 = nn.MaxPool2d((2, 2))
#         self.conv2 = nn.Conv2d()
#         self.conv1 = self.conv2DBlock(1024, 256, (5, 5))
#         self.conv2 = self.conv2DBlock(1024, 1024, (3, 3))
#         self.conv3 = self.conv2DBlock(1024, 512, (3, 3))
        self.fc1 = nn.Linear(1024, 1024)
        
        self.out = nn.Linear(1024, out_dim)
        
#     def conv2DBlock(self, in_filter, out_filter, kernel_size):
#         return nn.Sequential(
#             nn.Conv2d(in_filter, in_filter, kernel_size, padding="same"),
#             nn.Conv2d(in_filter, in_filter, kernel_size, padding="same"),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(1,1)),
#             nn.Conv2d(in_filter, out_filter, kernel_size, padding="valid")
#         )
    
    def forward(self, x):
        x = self.input(x) # in: n x 3 x 150 x 150   out: n x 1024 x 148 x 148
        x = self.conv1(x) # in: n x 1024 x 148 x 148  out: n x 512 x 146 x 146
        x = F.relu(self.maxp1(x)) # in: n x 512 x 146 x 146  out: n x 512 x 73 x 73
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
        x = x.view((-1, 128*73*73))
        x = F.relu(self.fc1(x)) # in: n x 2728448 out: 1024
#         x = F.relu(self.fc2(x))
        return F.softmax(self.out(x))

In [9]:
model = ClassificationModel(100)
model = model.to(device)
model

ClassificationModel(
  (input): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (conv1): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxp1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=1024, bias=True)
  (out): Linear(in_features=1024, out_features=100, bias=True)
)

In [10]:
class Trainer(object):
    def __init__(self, optimizer, criteria, epochs=10, scheduler=None):
        """
        This class will train the model based on the 
        - optimizer: Optimizer algorithm (object)
        - criteria: It is the loss function which will be used like CrossEntropyLoss, MSELoss etc.
        """
        self.optimizer = optimizer
        self.epochs = epochs
        self.criteria = criteria
        self.scheduler = scheduler
        
    
    def train_one_step(self, x, y):
        """
        Training on Single Step
        - Predict the output
        - Optimize the parameters
        """
        self.optimizer.zero_grad() # Initialization of Gredients to 0
        y_hat = self.model(x)
        loss = self.criteria(y_hat, y)
        loss.backward()
        self.optimizer.step()
        return loss.item()
    
    def train_one_epoch(self, data_loader):
        
        """
        This function will enable the epoch training and return the loss for the epoch
        """
        self.model.train() # Setting model in Training Mode
        total_loss = 0
        for idx, data in tqdm(enumerate(data_loader), total=len(data_loader)):
            loss = self.train_one_step(**data)
            data.detach()
            total_loss += loss
        return total_loss / (idx + 1)
    
    def eval_one_epoch(self, data_loader):
        """
        This function will enable the epoch training and return the loss for the epoch
        """
        self.model.eval() # Setting model in Evaluation Mode
        total_loss = 0
        for idx, data in enumerate(data_loader):
            x, y = data["x"], data["y"]
            y_hat = self.model(x)
            loss = self.criteria(y_hat, y)
            total_loss += loss.item()
        return total_loss / (idx + 1)
    
    def fit(self, model, train_loader, valid_loader=None, scheduler=None, **kwargs):
        """
        This function will start the model training. 
        """
        self.model = model
        valid_loss = None
        for epoch in range(self.epochs):
            loss = self.train_one_epoch(train_loader)
            if valid_loader:
                valid_loss = self.eval_one_epoch(valid_loader)
            if hasattr(self, "sechduler") and self.sechduler != None:
                self.scheduler.step()
            tqdm.write(f"Epoch: {epoch}, Training Loss: {loss}, Validation Loss: {valid_loss}")
        return self.model
    

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=1e-5)
criteria = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5)
model_trainer = Trainer(optimizer = optimizer, criteria = criteria, scheduler=scheduler, epochs=100)
model_trainer.fit(model = model, train_loader=train_loader, valid_loader=test_loader)

  0%|          | 0/25000 [00:00<?, ?it/s]

C:\Users\ritik\miniconda3\envs\gpu\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
<ipython-input-8-824a742236f1>:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.out(x))


RuntimeError: Function AddmmBackward returned an invalid gradient at index 1 - got [2, 1024] but expected shape compatible with [2, 682112]